In [ ]:
import pandas as pd
from astroquery.simbad import Simbad
from jwst import *

#assumes your target is in Tim Pearce's CSV File
target_name = 'HD61005'

df = pd.read_csv('../allDiscAnalysisResults.csv')
data_arr = df[['Target','Distance_pc','Age_Myr','PearceMPlt_MJup', 'PearcePltApocentre_arcsec']].to_numpy()
target_dict={}
for row in data_arr:
    target_dict[row[0]]=row[1:]

distance,age,planet_mass,separation = target_dict[target_name]

Query Simbad for Spectral type and K-mag

In [ ]:
Simbad.add_votable_fields('flux(K)','sp')
result_table = Simbad.query_object(target_name)
kmag,spt = result_table['FLUX_K'][0], result_table['SP_TYPE'][0]

Query all targets to and force spectral types

In [ ]:
targets=df['Target'].to_numpy()
queried_params=[];spts=[]
for target in targets:
    result_table = Simbad.query_object(target)
    spts.append(result_table['SP_TYPE'][0])
    queried_params.append([target,result_table['FLUX_K'][0]])

In [ ]:
def match_spt(spt,lookuptable=lookuptable):
    available_keys=list(lookuptable.keys())
    matches=[];diffs=[]
    for key in available_keys:
        if spt[0]==key[0]:
            matches.append(key)
    for match in matches:
        diffs.append(np.abs(int(spt[1])-int(match[1])))
    forced_spt= spt[0] + str(matches[np.where(diffs==np.min(diffs))[0][0]])[1]+'V'
    return forced_spt

spts[40]='A5V'
spts[103]='A1V'
for i,spt in enumerate(spts):
    if spt not in lookuptable: 
        spts[i] = match_spt(spt)
[params.append(spts[i]) for i, params in  enumerate(queried_params)]

#adding another dictionary to hold these queried paramters in a more useful format
simbad_vals={}
for star in queried_params:
    simbad_vals[star[0]]=[star[1],star[2]]


In [ ]:
instrument='nircam'
instrument_mask = 'mask335r'
jwst_filt = 'f444w'

kmag,spt = simbad_vals[target_name]

tst = get_jwst_mag(spt,kmag,instrument,jwst_filt,filter_dir="./",jwst_mask=instrument_mask,
                norm_filter='bessel_k')
print(tst)